In [1]:
import pdgstaging
from pdgstaging import TileStager
from pathlib import Path
import pdgraster
import geopandas as gpd

from datetime import datetime
import logging
import logging.config # commented out cause Robyn suggested that she uses diff approach for logging without parsl or ray
# she uses logging.basicConfig() in script/notebook instead
import argparse
import json

import config

In [2]:
# LOGGING:

logging_config = '/home/jcohen/viz-staging/logging.json' # commented out cause Robyn suggested that she uses diff approach

def setup_logging(log_json_file): # commented out cause Robyn suggested that she uses diff approach
    """
    Setup logging configuration
    """
    with open(log_json_file, 'r') as f:
        logging_dict = json.load(f)
    logging.config.dictConfig(logging_dict)
    return logging_dict

logging_dict = setup_logging(logging_config)

# from Robyn:
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("debug.log"),
#         logging.StreamHandler()
#     ]
# )

# TileStager({
#   "deduplicate_clip_to_footprint": True,
#   "dir_output": "/home/jcohen/viz-workflow/", 
#   "dir_input": "/home/jcohen/iwp_workflow_testing/input/russia_subset/", 
#   "ext_input": ".shp",
#   "ext_footprints": ".shp",
#   "dir_footprints": "/home/jcohen/iwp_workflow_testing/footprints/russia_subset/", 
#   "dir_staged": "staged/",
#   "dir_geotiff": "geotiff/", 
#   "dir_web_tiles": "web_tiles/", 
#   "filename_staging_summary": "staging_summary.csv",
#   "filename_rasterization_events": "raster_events.csv",
#   "filename_rasters_summary": "raster_summary.csv",
#   "filename_config": "config",
#   "simplify_tolerance": 0.1,
#   "tms_id": "WGS1984Quad",
#   "z_range": [
#     0,
#     15
#   ],
#   "geometricError": 57,
#   "z_coord": 0,
#   "statistics": [
#     {
#       "name": "iwp_coverage",
#       "weight_by": "area",
#       "property": "area_per_pixel_area",
#       "aggregation_method": "sum",
#       "resampling_method": "average",
#       "val_range": [
#         0,
#         1
#       ],
#       "palette": [
#         "#66339952",
#         "#ffcc00"
#       ],
#       "nodata_val": 0,
#       "nodata_color": "#ffffff00"
#     },
#   ],
#   "deduplicate_at": [
#     "staging"
#   ],
#   "deduplicate_keep_rules": [
#     [
#       "Date",
#       "larger"
#     ]
#   ],
#   "deduplicate_method": "footprints"
# }).stage_all()

IMPORT DATA:

In [3]:
# russia subset processed in viz-workflow issue#19: check of clipping to footprint is working properly
input_shp = "/home/jcohen/iwp_workflow_testing/input/russia_subset/WV02_20100720235408_1030010006ABA200_10JUL20235408-M1BS-500152191020_01_P002_u16rf3413_pansh.shp"
footprint_filepath = "/home/jcohen/iwp_workflow_testing/footprints/russia_subset/WV02_20100720235408_1030010006ABA200_10JUL20235408-M1BS-500152191020_01_P002_u16rf3413_pansh.shp"

CONFIGURE STAGER AND MANAGERS

In [4]:
config = config.IWP_CONFIG
stager = pdgstaging.TileStager(config = config)
tile_manager = stager.tiles
config_manager = stager.config

Modify the stager.stage() approach in order to integrate logging.

In [5]:
def custom_stage(path, config, logging_dict = logging_dict): 
    """
    Stage files (step 1)
    """
    import pdgstaging
    if logging_dict:
        import logging.config
        logging.config.dictConfig(logging_dict)
    stager = pdgstaging.TileStager(config)
    stager.stage(path)
    return True

STAGE THE INPUT SHP FILE

In [6]:
custom_stage(path = input_shp, config = config, logging_dict = logging_dict)
#stager.stage(path = input_shp)
# output files will have the staging_duplicated property

KeyboardInterrupt: 